## 【復習】配列操作

In [97]:
a = reshape(1:6 , (2 , 3))

2×3 reshape(::UnitRange{Int64}, 2, 3) with eltype Int64:
 1  3  5
 2  4  6

In [98]:
a[1, 3]

5

In [99]:
a[2 , 2]

4

In [100]:
size(a)

(2, 3)

In [101]:
size(a , 1) #行成分のサイズを取得

2

In [102]:
size(a , 2) #列成分のサイズを取得

3

# 行列処理の高速化

行列の要素を順に処理する際に，Julia では列方向に処理したほうが効率が良い．(列優先(column-major))

In [103]:
function sum_by_col(x)
    #値の初期化(以下のように初期化することで型が安定する)
    s = zero(eltype(x))
    for j in 1:size(x , 2) #size(x , 2)で列成分のサイズを取得
        for i in size(x , 1) #size(x , 1)で行成分のサイズを取得
            s += x[i , j]
        end
    end
    s
end

sum_by_col (generic function with 1 method)

In [104]:
function sum_by_row(x)
    s = zero(eltype(x))
    for i in 1:size(x , 1)
        for j in 1:size(x , 2)
            s += x[i , j]
        end
    end
    s
end

sum_by_row (generic function with 1 method)

In [105]:
using BenchmarkTools

In [106]:
#末尾に `;` を追加することで結果が表示されないようにしている
x = rand(10^4 , 10^4);

In [107]:
@btime sum_by_col(x)

  100.517 μs (1 allocation: 16 bytes)


4983.038335833042

In [108]:
@btime sum_by_row(x)

  942.017 ms (1 allocation: 16 bytes)


4.99999390376198e7

上記より，列方向への計算が遥かに高速である．返す値も微妙に異なる．

In [109]:
sum_by_col(x)

4983.038335833042

In [110]:
sum_by_row(x)

4.99999390376198e7

# 条件文のあるループの効率的な実行

In [111]:
using Random

In [112]:
Random.seed!(1);

In [113]:
#標準正規分布に従う乱数
x = randn(10^6)

1000000-element Array{Float64,1}:
  0.2972879845354616 
  0.3823959677906078 
 -0.5976344767282311 
 -0.01044524463737564
 -0.839026854388764  
  0.31111133849833383
  2.2950878238373105 
 -2.2670863488005306 
  0.5299655761667461 
  0.43142152642291204
  0.5837082875687786 
  0.9632716050381906 
  0.45879095505371686
  ⋮                  
 -0.25610515720884475
  1.3848574512180376 
 -0.9790418271457938 
 -0.06984751088684674
  0.32029025213894724
 -0.460577128909139  
  0.26085911346274276
  0.5635158479663883 
  0.6418951662098386 
  0.06143017075770961
 -2.058652918548743  
  1.004460436892121  

いくつかの配列中の正の値の和を求める方法を示し，性能を比較する．

In [114]:
@btime sum(v for v in x if v > 0)

  4.628 ms (8 allocations: 128 bytes)


398244.60749279766

In [115]:
function possum1(x)
    s = zero(eltype(x))
    for v in x
        if v > 0
            s += v
        end
    end
    s
end

possum1 (generic function with 1 method)

In [116]:
@btime possum1(x)

  1.787 ms (1 allocation: 16 bytes)


398244.60749279766

In [117]:
function possum2a(x)
    s = zero(eltype(x))
    for v in x
        s += ifelse(v > 0 , v , zero(s))
    end
    s
end

possum2a (generic function with 1 method)

In [118]:
@btime possum2a(x)

  1.215 ms (1 allocation: 16 bytes)


398244.60749279766

In [119]:
function possum2b(x)
    s = zero(eltype(x))
    @simd for v in x
        s += ifelse(v > 0 , v , zero(s))
    end
    s
end

possum2b (generic function with 1 method)

In [120]:
@btime possum2b(x)

  187.238 μs (1 allocation: 16 bytes)


398244.6074928036

In [121]:
function possum2c(x)
    s = 0
    for v in x
        s += ifelse(v > 0 , v , 0)
    end
    s
end

possum2c (generic function with 1 method)

In [122]:
@btime possum2c(x)

  6.145 ms (1 allocation: 16 bytes)


398244.60749279766

# Reference

In [123]:
?Random.seed!()

```
seed!([rng=GLOBAL_RNG], seed) -> rng
seed!([rng=GLOBAL_RNG]) -> rng
```

Reseed the random number generator: `rng` will give a reproducible sequence of numbers if and only if a `seed` is provided. Some RNGs don't accept a seed, like `RandomDevice`. After the call to `seed!`, `rng` is equivalent to a newly created object initialized with the same seed.

If `rng` is not specified, it defaults to seeding the state of the shared thread-local generator.

# Examples

```julia-repl
julia> Random.seed!(1234);

julia> x1 = rand(2)
2-element Array{Float64,1}:
 0.590845
 0.766797

julia> Random.seed!(1234);

julia> x2 = rand(2)
2-element Array{Float64,1}:
 0.590845
 0.766797

julia> x1 == x2
true

julia> rng = MersenneTwister(1234); rand(rng, 2) == x1
true

julia> MersenneTwister(1) == Random.seed!(rng, 1)
true

julia> rand(Random.seed!(rng), Bool) # not reproducible
true

julia> rand(Random.seed!(rng), Bool)
false

julia> rand(MersenneTwister(), Bool) # not reproducible either
true
```


In [124]:
? randn

search: randn randn! randstring rand rand! randexp Random randperm randexp!



```
randn([rng=GLOBAL_RNG], [T=Float64], [dims...])
```

Generate a normally-distributed random number of type `T` with mean 0 and standard deviation 1. Optionally generate an array of normally-distributed random numbers. The `Base` module currently provides an implementation for the types [`Float16`](@ref), [`Float32`](@ref), and [`Float64`](@ref) (the default), and their [`Complex`](@ref) counterparts. When the type argument is complex, the values are drawn from the circularly symmetric complex normal distribution of variance 1 (corresponding to real and imaginary part having independent normal distribution with mean zero and variance `1/2`).

# Examples

```jldoctest
julia> using Random

julia> rng = MersenneTwister(1234);

julia> randn(rng, ComplexF64)
0.6133070881429037 - 0.6376291670853887im

julia> randn(rng, ComplexF32, (2, 3))
2×3 Array{Complex{Float32},2}:
 -0.349649-0.638457im  0.376756-0.192146im  -0.396334-0.0136413im
  0.611224+1.56403im   0.355204-0.365563im  0.0905552+1.31012im
```


In [125]:
?eltype

search: eltype fieldtype fieldtypes



```
eltype(type)
```

Determine the type of the elements generated by iterating a collection of the given `type`. For dictionary types, this will be a `Pair{KeyType,ValType}`. The definition `eltype(x) = eltype(typeof(x))` is provided for convenience so that instances can be passed instead of types. However the form that accepts a type argument should be defined for new types.

# Examples

```jldoctest
julia> eltype(fill(1f0, (2,2)))
Float32

julia> eltype(fill(0x1, (2,2)))
UInt8
```


In [126]:
? sum

search: sum sum! summary sum_by_row sum_by_col cumsum cumsum! isnumeric possum3



```
sum(f, itr)
```

Sum the results of calling function `f` on each element of `itr`.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

# Examples

```jldoctest
julia> sum(abs2, [2; 3; 4])
29
```

Note the important difference between `sum(A)` and `reduce(+, A)` for arrays with small integer eltype:

```jldoctest
julia> sum(Int8[100, 28])
128

julia> reduce(+, Int8[100, 28])
-128
```

In the former case, the integers are widened to system word size and therefore the result is 128. In the latter case, no such widening happens and integer overflow results in -128.

---

```
sum(itr)
```

Returns the sum of all elements in a collection.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

# Examples

```jldoctest
julia> sum(1:20)
210
```

---

```
sum(A::AbstractArray; dims)
```

Sum elements of an array over the given dimensions.

# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> sum(A, dims=1)
1×2 Array{Int64,2}:
 4  6

julia> sum(A, dims=2)
2×1 Array{Int64,2}:
 3
 7
```


In [127]:
? ifelse

search: ifelse



```
ifelse(condition::Bool, x, y)
```

Return `x` if `condition` is `true`, otherwise return `y`. This differs from `?` or `if` in that it is an ordinary function, so all the arguments are evaluated first. In some cases, using `ifelse` instead of an `if` statement can eliminate the branch in generated code and provide higher performance in tight loops.

# Examples

```jldoctest
julia> ifelse(1 > 2, 1, 2)
2
```


In [128]:
? @simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

  * It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
  * Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

  * The loop must be an innermost loop
  * The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is   currently needed for all array accesses. The compiler can sometimes turn   short `&&`, `||`, and `?:` expressions into straight-line code if it is safe   to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)   function instead of `?:` in the loop if it is safe to do so.
  * Accesses must have a stride pattern and cannot be "gathers" (random-index   reads) or "scatters" (random-index writes).
  * The stride should be unit stride.

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:


  * There exists no loop-carried memory dependencies
  * No iteration ever waits on a previous iteration to make forward progress.
